In [ ]:
# 🔁 Caricamento moduli
import sys
sys.path.append("../src")

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

from src.dataset_loader import get_dataloader  # carica train
import os

# ⚙️ Impostazioni
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"💻 Using device: {device}")

# 📦 Dataloader
train_path = "/content/dataset_prepared/train" if "COLAB_GPU" in os.environ else "datasets/dataset_prepared/train"
dataset, dataloader = get_dataloader(data_dir=train_path, batch_size=32)

# 🧠 Modello ResNet18 pre-addestrato
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # 2 classi: IDC / non-IDC
model = model.to(device)

# 🔧 Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 🔁 Training loop
EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    print(f"📈 Epoch {epoch+1}/{EPOCHS} - Loss: {avg_loss:.4f}")

# 💾 Salva il modello
os.makedirs("results", exist_ok=True)
torch.save(model.state_dict(), "results/histology_model.pth")
print("✅ Model saved to results/histology_model.pth")
